DEC 알고리즘을 대전시 자료를 활용할 수 있도록 수정한 파일
입력자료: 44개 검지기, 1일 16시간, 10분간격, 5주간 입력 자료, 교통량과 속도자료 

In [1]:
from keras.datasets import mnist
import numpy as np
np.random.seed(10)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 80% !important; }</style>"))

Using TensorFlow backend.


In [2]:
from time import time
import numpy as np
import tensorflow as tf
import keras
#import tensorflow.keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import pandas as pd
import metrics
import warnings
from keras import backend as K

In [3]:
print(keras.__version__)
import warnings
warnings.filterwarnings( 'ignore' )


tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

2.0.5


False

In [4]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)
    
    # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

In [5]:
## 입력자료를 확인하고 input format 수정

trains = np.loadtxt("C:/Users/KOTI/Documents/2020 박용진/1. 자료분석/계백로/input_normalize.csv",
                    skiprows=1, delimiter=',', dtype=float)

x = trains[:, 267:531].reshape([len(trains), 264])  # 열의 갯수를 수정
y_year = trains[:, 0].reshape([len(trains)])
y_weekday = trains[:, 1].reshape([len(trains)])
y = trains[:, 2].reshape([len(trains)])

x = x.reshape((x.shape[0], -1))
#x = np.divide(x, 256.)        #normalized 데이터에서는 삭제


In [6]:
n_clusters = 5 #len(np.unique(y))  # cluster의 갯수를 직접 선언해야 함
x.shape
print ([y_year, y_weekday, y])

[array([20190818., 20190818., 20190818., ..., 20190921., 20190921.,
       20190921.]), array([1., 1., 1., ..., 7., 7., 7.]), array([ 600.,  610.,  620., ..., 2130., 2140., 2150.])]


In [7]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20, n_jobs=4)
y_pred_kmeans = kmeans.fit_predict(x)

In [8]:
metrics.acc(y, y_pred_kmeans)  # 우리 자료에는 label이 없으므로 accuracy를 구할 수 없음

AttributeError: module 'metrics' has no attribute 'acc'

In [9]:
#dims = [x.shape[-1], 500, 500, 2000, 10]
dims = [x.shape[-1], 500, 1000, 2000, 3000, 5000, 10]   # hidden layer를 어떻게 구성할 것인지 고민 
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=0.1, momentum=0.9)
pretrain_epochs = 1000
batch_size = 256
save_dir = './results'

In [10]:
autoencoder, encoder = autoencoder(dims, init=init)

In [ ]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)
autoencoder.save_weights(save_dir + '/ae_weights.h5')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/1000




3360/3360 [==============================] - 7s - loss: 0.1005     
Epoch 2/1000
3360/3360 [==============================] - 5s - loss: 0.0855     
Epoch 3/1000
3360/3360 [==============================] - 5s - loss: 0.0680     
Epoch 4/1000
3360/3360 [==============================] - 5s - loss: 0.0512     
Epoch 5/1000
3360/3360 [==============================] - 5s - loss: 0.0356     
Epoch 6/1000
3360/3360 [==============================] - 5s - loss: 0.0223     
Epoch 7/1000
3360/3360 [==============================] - 5s - loss: 0.0136     
Epoch 8/1000
3360/3360 [==============================] - 5s - loss: 0.0107     
Epoch 9/1000
3360/3360 [==============================] - 5s - loss: 0.0102     
Epoch 10/1000
3360/3360 [==============================] - 5s - loss: 0.0101     
Epoch 11/1000
3360/3360 [==============================] - 5s - 

In [ ]:
autoencoder.load_weights(save_dir + '/ae_weights.h5')

In [35]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape = (self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [36]:
clustering_layer = ClusteringLayer(n_clusters, name ='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

loss = 0
index = 0
maxiter = 20000
update_interval = 140
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

In [37]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(metrics.acc(y, y_pred), 5)
            nmi = np.round(metrics.nmi(y, y_pred), 5)
            ari = np.round(metrics.ari(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/DEC_model_final.h5')

Iter 0: acc = 0.04613, nmi = 0.35613, ari = 0.02960  ; loss= 0
Iter 140: acc = 0.04643, nmi = 0.34686, ari = 0.03027  ; loss= 0.0259
Iter 280: acc = 0.04673, nmi = 0.33765, ari = 0.03041  ; loss= 0.03754
Iter 420: acc = 0.04524, nmi = 0.33077, ari = 0.02963  ; loss= 0.05804
Iter 560: acc = 0.04405, nmi = 0.32464, ari = 0.02894  ; loss= 0.07754
Iter 700: acc = 0.04345, nmi = 0.32005, ari = 0.02843  ; loss= 0.09309
Iter 840: acc = 0.04226, nmi = 0.31808, ari = 0.02819  ; loss= 0.1031
Iter 980: acc = 0.04286, nmi = 0.31423, ari = 0.02727  ; loss= 0.11036
Iter 1120: acc = 0.04196, nmi = 0.32956, ari = 0.02944  ; loss= 0.10942
Iter 1260: acc = 0.02292, nmi = 0.14111, ari = 0.00105  ; loss= 1.16853
Iter 1400: acc = 0.02411, nmi = 0.14359, ari = 0.00105  ; loss= 0.01224
Iter 1540: acc = 0.02440, nmi = 0.14432, ari = 0.00107  ; loss= 0.01323
Iter 1680: acc = 0.02411, nmi = 0.14440, ari = 0.00107  ; loss= 0.01487
Iter 1820: acc = 0.02381, nmi = 0.14425, ari = 0.00107  ; loss= 0.0146
Iter 1960: 

In [16]:
model.load_weights(save_dir + '/DEC_model_final.h5')

In [17]:
# Eval.
############ redefine test data set ##############

test = np.loadtxt("C:/Users/KOTI/Documents/2020 박용진/1. 자료분석/계백로/input_raw.csv", skiprows=1, delimiter=',', dtype=float)
x = test[:, 267:531].reshape([len(test), 264])

y_year = test[:, 0].reshape([len(test)])
y_weekday = test[:, 1].reshape([len(test)])
y = test[:, 2].reshape([len(test)])

x = x.reshape((x.shape[0], -1))
#x = np.divide(x, 255.)    #normalized 데이터에서는 삭제


q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    acc = np.round(metrics.acc(y, y_pred), 5)
    nmi = np.round(metrics.nmi(y, y_pred), 5)
    ari = np.round(metrics.ari(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('Acc = %.5f, nmi = %.5f, ari = %.5f' % (acc, nmi, ari), ' ; loss=', loss)

Acc = 0.04048, nmi = 0.26889, ari = 0.02213  ; loss= 0.03568


In [18]:
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt
sns.set(font_scale=3)
confusion_matrix = sklearn.metrics.confusion_matrix(y, y_pred)

plt.figure(figsize=(16, 14))
sns.heatmap(confusion_matrix, annot=True, fmt="d", annot_kws={"size": 20});
plt.title("Confusion matrix", fontsize=30)
plt.ylabel('True label', fontsize=25)
plt.xlabel('Clustering label', fontsize=25)
plt.show()

<Figure size 1600x1400 with 2 Axes>

In [19]:
from sklearn.utils.linear_assignment_ import linear_assignment

y_true = y.astype(np.int64)
D = max(y_pred.max(), y_true.max()) + 1
w = np.zeros((D, D), dtype=np.int64)
# Confusion matrix.
for i in range(y_pred.size):
    w[y_pred[i], y_true[i]] += 1
ind = linear_assignment(-w)

sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

0.04047619047619048

In [20]:
y_pred = y_pred.reshape(-1, 1)
y = y.reshape(-1, 1)

df = pd.DataFrame(y_year, columns = ['year'])
df['weekday'] = y_weekday
df['true'] = y
df['cluster_id'] = y_pred
df.to_excel('C:/Users/KOTI/project/AI_traffic_management/Deep Clustering/DenseLayer/results/test.xlsx', index = False )